In [1]:
from sympy import *
__VERSION__ = "3.1+margp2+TODO"
__DATE__ = "12/Dec/2022"

# Carbon Calculations

Various calculations related to the Carbon whitepaper.

In [137]:
def isolate(eqn, var,ix=0):
    "isolates the variable var in the equation eqn and returns result as equation"
    return Eq(var, solve(eqn, var)[ix])
def subs(srceq, subeq):
    "substitutes srceq with the relation in subeq (subeq eg `y=x**2`)"
    return srceq.subs(subeq.lhs, subeq.rhs).simplify()

## Core Formulas

In [3]:
x,x0,xint,dx,Dx = symbols("x x_0 x_{int} dx \Delta{x}")
y,y0,yint,dy,Dy = symbols("y y_0 y_{int} dy \Delta{y}")
P,Pmarg,Px,Py,Q,Gam,w,B,S = symbols("P_0 P_{marg} P_x P_y Q \Gamma w B S")
Pa = Py
Pb = Px
xasym, yasym, kappa, k = symbols("x_{asym} y_{asym} \kappa k")
P0=P

This is the **Pool Invariant Equation** in its canonic **risk asset** form using $P, x_0, \Gamma$

In [4]:
PIE = Eq(
    y,
    x0*P*(x*(Gam - 1) - x0*(Gam - 2))/(Gam*x - x0*(Gam - 1))
)
PIE

Eq(y, P_0*x_0*(x*(\Gamma - 1) - x_0*(\Gamma - 2))/(\Gamma*x - x_0*(\Gamma - 1)))

In [5]:
PINE = PIE.subs(x0, y0/P).simplify()
PINE

Eq(y, y_0*(P_0*x*(\Gamma - 1) - y_0*(\Gamma - 2))/(P_0*\Gamma*x - y_0*(\Gamma - 1)))

This is the **Q-form** of the **Pool Invariant Equation**

In [6]:
PIQE = Eq(
    Q,
    x*y / ( (x-xint)*(y-yint) )
)
PIQE

Eq(Q, x*y/((x - x_{int})*(y - y_{int})))

In [7]:
_m = (x-xint)*(y-yint)
PIQ2E = Eq(PIQE.lhs*_m, PIQE.rhs*_m)
PIQ2E

Eq(Q*(x - x_{int})*(y - y_{int}), x*y)

This is the **Definition Equations** for $Q$ in terms of $\Gamma$ and its inverse

In [120]:
QGDE = Eq(Q,(1-Gam)**2)
QDE = QGDE
QGDE

Eq(Q, (1 - \Gamma)**2)

In [9]:
GQE = isolate(QGDE, Gam)
GQE

Eq(\Gamma, 1 - sqrt(Q))

Those are useful identities between a certain expression of $\Gamma$ and certain expressions of $Q$ and vice versa

In [10]:
GFQE = Eq((2-Gam)/(1-Gam),(1+1/sqrt(Q)))
GFQE

Eq((2 - \Gamma)/(1 - \Gamma), 1 + 1/sqrt(Q))

In [11]:
(GFQE.lhs-GFQE.rhs).subs(Gam, GQE.rhs).simplify()

0

For the below, note that $Q=\sqrt[4]{w}$, ie the fourth root of the width of the range

In [12]:
GFQ2E = Eq((Gam-1)/Gam, ((Gam-1)/Gam).subs(Gam, GQE.rhs).simplify())
GFQ2E

Eq((\Gamma - 1)/\Gamma, sqrt(Q)/(sqrt(Q) - 1))

## Other Parametrizations

Those are the **Definition Equations** for $x_{int}, y_{int}, P_x, P_y$

In [13]:
XiDE = Eq(xint, x0*(2-Gam)/(1-Gam))
XiDE

Eq(x_{int}, x_0*(2 - \Gamma)/(1 - \Gamma))

In [14]:
XiQE = XiDE.subs(GFQE.lhs, GFQE.rhs)
XiQE

Eq(x_{int}, x_0*(1 + 1/sqrt(Q)))

In [15]:
YiDE = Eq(yint, y0*(2-Gam)/(1-Gam))
YiDE

Eq(y_{int}, y_0*(2 - \Gamma)/(1 - \Gamma))

In [16]:
YiQE = YiDE.subs(GFQE.lhs, GFQE.rhs)
YiQE

Eq(y_{int}, y_0*(1 + 1/sqrt(Q)))

In [17]:
PxDE = Eq(Px, P*Q)
PxDE

Eq(P_x, P_0*Q)

In [18]:
PyDE = Eq(Py, P/Q)
PyDE

Eq(P_y, P_0/Q)

Here are a few other important relationships. Firstly, $P_0$ is the ratio the reference coordinates

In [121]:
PXYE = Eq(P, y0/x0)
P0DE = PXYE
PXYE

Eq(P_0, y_0/x_0)

and more importantly also of the intercepts

In [20]:
PXYiE = Eq(P, yint/xint)
PXYiE

Eq(P_0, y_{int}/x_{int})

$Q$ is related to the ratio of the intercept prices (smaller value in numerator)

In [21]:
QPE = Eq(Q, sqrt(Px/Py))
QPE

Eq(Q, sqrt(P_x/P_y))

In fact, if we define $w>1$ as the (percentage) **range width** (bigger value in numerator)

In [22]:
WDE = Eq(w, 1/QPE.rhs**2)
WDE

Eq(w, P_y/P_x)

then that width is simply the inverse Q squared

In [23]:
WE = Eq(w, 1/QPE.lhs**2)
WE

Eq(w, Q**(-2))

In [24]:
WE.lhs.subs(w, WDE.rhs)-WE.rhs.subs(Q, QPE.rhs)

0

Also the reference price is the geometric average of the intercept prices

In [25]:
PPE = Eq(P,sqrt(Px*Py))
PPE

Eq(P_0, sqrt(P_x*P_y))

Below formulas are important for actually setting ranges for unidirectional trading where the parameters we have are the amount of risk asset $x_{int}$ as well as the two ends of the range $P_x, P_y$. Our goal is to solve for the parameterset $Q, x_{int}, y_{int}$ that allows us to use the invariant equation `PIQE`. $x_{int}$ is already known, and $Q$ we can get from `QPE`, so what is left is to calculate $y_{int}$.

We first recall a number of definitions

In [26]:
QPE

Eq(Q, sqrt(P_x/P_y))

In [27]:
PPE

Eq(P_0, sqrt(P_x*P_y))

In [28]:
PXYiE

Eq(P_0, y_{int}/x_{int})

and now make a number of new ones

In [29]:
YiPE = Eq(yint, solve(PXYiE, yint)[0].subs(P, PPE.rhs))
YiPE

Eq(y_{int}, x_{int}*sqrt(P_x*P_y))

In [30]:
XiPE = Eq(xint, solve(YiPE, xint)[0])
XiPE

Eq(x_{int}, y_{int}/sqrt(P_x*P_y))

This is the **Swap Equation** derived from it, describing an actual transaction

In [31]:
SE = Eq(
    - Dy,
    P*x0**2*Dx/((Gam*(x - x0) + x0)*(Gam*(x - x0 + Dx) + x0))
)
SE

Eq(-\Delta{y}, P_0*\Delta{x}*x_0**2/((\Gamma*(x - x_0) + x_0)*(\Gamma*(\Delta{x} + x - x_0) + x_0)))

This is the **Marginal Swap Equation** describing an infinitesimal transaction, rearranged with $dx$ on the LHS 

In [32]:
MSE = Eq(
    -dy/dx,
    P*x0**2/(Gam*(x - x0) + x0)**2
)
MSE

Eq(-dy/dx, P_0*x_0**2/(\Gamma*(x - x_0) + x_0)**2)

This is the **Marginal Price Equation** which is the same as the Marginal Swap Equation, just recognising that $-\frac{dy}{dx}=P_{marg}%$

In [33]:
MPE = Eq(Pmarg, MSE.rhs)
MPE

Eq(P_{marg}, P_0*x_0**2/(\Gamma*(x - x_0) + x_0)**2)

This is the **Reverse Marginal Price Equation** with $y,y_0$ instead of $x,x_0$, but with the price parameters unchanged

In [34]:
MPERev0 = MPE.subs(Pmarg, 1/Pmarg).subs(P,1/P).subs(x0,y0).subs(x,y)
MPERev0

Eq(1/P_{marg}, y_0**2/(P_0*(\Gamma*(y - y_0) + y_0)**2))

The is again the **Reverse Marginal Price Equation** but inversed to isolate $P_{marg}$

In [35]:
MPERev = Eq(Pmarg, 1/MPERev0.rhs)
MPERev

Eq(P_{marg}, P_0*(\Gamma*(y - y_0) + y_0)**2/y_0**2)

Now we reparameterize the MPE as a function of $Q, x_{int}$ and of $Q, y_{int}$

In [36]:
MPQXiE = MPE.subs(GQE.lhs,GQE.rhs).subs(x0, solve(XiQE, x0)[0]).simplify()
MPQXiE

Eq(P_{marg}, P_0*Q*x_{int}**2/(-Q*x + Q*x_{int} + x)**2)

In [37]:
MPQYiE = MPERev.subs(GQE.lhs,GQE.rhs).subs(y0, solve(YiQE, y0)[0]).simplify()
MPQYiE

Eq(P_{marg}, P_0*(-Q*y + Q*y_{int} + y)**2/(Q*y_{int}**2))

### Asymptotic form of the invariant equation and scaling

The invariant equation is a hyperbola with the asymptotes $x_{asym}, y_{asym}$. Note that in the formulas below for $0<\Gamma<1$ we find that $1-1/\Gamma<0$, meaning the asymptotes are negative values

In [38]:
PIA1E = Eq( (x-xasym)*(y-yasym),kappa)
PIA1E

Eq((x - x_{asym})*(y - y_{asym}), \kappa)

In [39]:
XADE = Eq(xasym, x0*(1-1/Gam))
XADE

Eq(x_{asym}, x_0*(1 - 1/\Gamma))

In [40]:
YADE = Eq(yasym, y0*(1-1/Gam))
YADE

Eq(y_{asym}, y_0*(1 - 1/\Gamma))

In [41]:
YAXDE = Eq(yasym, (y0*(1-1/Gam)).subs(y0, P0*x0))
YAXDE

Eq(y_{asym}, P_0*x_0*(1 - 1/\Gamma))

In the above invariant equation we can substitute the above definitions of the quantities, and we substitute $y$ from the original pool invariant equation. This allows us to compute $\kappa$

In [42]:
KPDE = PIA1E.subs(xasym, XADE.rhs).subs(yasym, YAXDE.rhs).subs(y, PIE.rhs).simplify()
KPDE

Eq(\kappa, P_0*x_0**2/\Gamma**2)

We remind ourselves that $P_0x_0^2=x_0y_0$ which yields the equation below. 

In [43]:
KPYDE = Eq(KPDE.lhs,KPDE.rhs.subs(P0, y0/x0).simplify())
KPYDE

Eq(\kappa, x_0*y_0/\Gamma**2)

This is important, because we remember that the constant-product equation can be written as $xy=k=x_0y_0$. We also remind ourselves that $\sqrt k$ is financially meaningful, and therefore $1/\Gamma$ can be interpreted as pool leverage factor.

In [44]:
PLE = Eq(kappa, k/Gam**2)
PLE

Eq(\kappa, k/\Gamma**2)

The scaling factor $1/\Gamma$ can be written as function of the pool width $P_y/P_x$ as

In [45]:
GSE = Eq(sqrt(kappa)/sqrt(k), (1/Gam).subs(Gam, GQE.rhs).subs(Q, QPE.rhs))
GSE

Eq(sqrt(\kappa)/sqrt(k), 1/(1 - (P_x/P_y)**(1/4)))

### Rho and r parametrizations

In [46]:
r, rho =symbols("r \\rho") 
RXE = Eq(x0, r*x)
RXIE = Eq(x, rho*x0)
RXE

Eq(x_0, r*x)

In [47]:
RXIE

Eq(x, \rho*x_0)

In [48]:
MPRE = MPE.subs(RXE.lhs, RXE.rhs).simplify()
MPRE

Eq(P_{marg}, P_0*r**2/(\Gamma*(r - 1) - r)**2)

In [49]:
MPRIE = MPE.subs(RXIE.lhs, RXIE.rhs).simplify()
MPRIE

Eq(P_{marg}, P_0/(\Gamma*(\rho - 1) + 1)**2)

For completeness, here also the invariant and swap equations using $r,\rho$

In [50]:
PIRE = PIE.subs(RXE.lhs, RXE.rhs).simplify()
PIRE

Eq(y, -P_0*r*x*(-\Gamma + r*(\Gamma - 2) + 1)/(\Gamma - r*(\Gamma - 1)))

In [51]:
PIRIE = PIE.subs(RXIE.lhs, RXIE.rhs).simplify()
PIRIE

Eq(y, P_0*x_0*(-\Gamma + \rho*(\Gamma - 1) + 2)/(\Gamma*\rho - \Gamma + 1))

In [52]:
SRE = SE.subs(RXE.lhs, RXE.rhs).simplify()
SRE

Eq(\Delta{y}, P_0*\Delta{x}*r**2*x/((\Gamma*(r - 1) - r)*(\Gamma*(\Delta{x} - r*x + x) + r*x)))

In [53]:
SRIE = Eq(SE.lhs, P*Dx*x0 / ((Gam*(rho-1)+1)*(Gam*(Dx+x0*(rho-1))+x0)))
SRIE

Eq(-\Delta{y}, P_0*\Delta{x}*x_0/((\Gamma*(\Delta{x} + x_0*(\rho - 1)) + x_0)*(\Gamma*(\rho - 1) + 1)))

In [54]:
(SRIE.rhs - SE.rhs.subs(RXIE.lhs, RXIE.rhs)).simplify()

0

### B,S parameterization

The B,S parameterization is not particular intuitive, but the formulas are well suited for implementation

In [55]:
BDE = Eq(B,sqrt(Pb))
BDE

Eq(B, sqrt(P_x))

In [56]:
SDE = Eq(S,sqrt(Pa)-sqrt(Pb))
SDE

Eq(S, -sqrt(P_x) + sqrt(P_y))

In [116]:
PxDE

Eq(P_x, P_0*Q)

In [131]:
isolate(PxDE,P0)

Eq(P_0, P_x/Q)

In [122]:
PxDE

Eq(P_x, P_0*Q)

In [123]:
P0DE

Eq(P_0, y_0/x_0)

In [124]:
QDE

Eq(Q, (1 - \Gamma)**2)

In [126]:
GQE

Eq(\Gamma, 1 - sqrt(Q))

In [141]:
YiQE

Eq(y_{int}, y_0*(1 + 1/sqrt(Q)))

In [145]:
PIE0 = subs(PIE, isolate(P0DE, x0))
PIE1 = subs(PIE0, GQE)
PIE2 = subs(PIE1, isolate(PxDE,P0))
#PIE2 = PIE1.subs()
PIE2

Eq(y, y_0*(P_x*sqrt(Q)*x - Q**(3/2)*y_0 - Q*y_0)/(P_x*sqrt(Q)*x - P_x*x - Q**(3/2)*y_0))

In [ ]:
PIE.rsh.subs()

Below is the **Swap Equation** (ie $\Delta y$ as a function of $\Delta x$) in B,S parameterization.

In [178]:
BSSE = Eq(Dy, Dx*(S*y + B*yint)**2 / (S*Dx * (S*y + B*yint) + yint**2))
BSSE

Eq(\Delta{y}, \Delta{x}*(B*y_{int} + S*y)**2/(S*\Delta{x}*(B*y_{int} + S*y) + y_{int}**2))

Here is the **Reverse Swap Equation** (ie $\Delta x$ as a function of $\Delta y$)

In [176]:
BSRSE = Eq(Dx, Dy * yint**2 / ((S*y + B*yint) * (S*y + B*yint - S*Dy)))
BSRSE

Eq(\Delta{x}, \Delta{y}*y_{int}**2/((B*y_{int} + S*y)*(B*y_{int} - S*\Delta{y} + S*y)))

The **marginal price equation** (where $P_{marg} = -dy/dx$) then becomes

In [57]:
MPBSE = Eq(Pmarg, (B+S*y/yint)**2)
MPBSE

Eq(P_{marg}, (B + S*y/y_{int})**2)

We check that the marginal price equation is consistent with the swap equations

In [180]:
((BSSE.rhs / Dx).subs(Dx,0)-MPBSE.rhs).simplify()

0

In [184]:
((BSRSE.rhs / Dy).subs(Dy,0) - 1/MPBSE.rhs).simplify()

0

In [185]:
YMPE = isolate(MPBSE,y)
YMPE

Eq(y, y_{int}*(-B + sqrt(P_{marg}))/S)

### Adjusting active ranges

As a reminder, and _inactive_ range is a range where the current market price is outside the range, and an _active_ range is one where it is inside. Active ranges split into _active active_ and _inactive active_ ranges. In the former, the market price equals the marginal price of the curve, in the latter the marginal price is somewhat off (in the direction where the marginal price shown in the direction of the AMM is _worse_ than that of the market, so it won't trade.

An active active AMM whose curve is to be readjusted to remain active active has only one remaining degree of freedom -- one can choose either width or mid price, but not both. One can of course change the parameters such that it turns inactive active, in which case the parameters space is two dimensional (width and mid-price), but there is a non-trivial boundary condition to be taken into account because one is starting from the boundary. When adjusting an inactive inactive range, the space is locally fully 2-dimensional, but at one point the boundary must be taken into account.

In a first step, we express our marginal price as a function of $P_0=\sqrt{P_xP_y}$ and $w=P_y/P_x$

In [59]:
MPWYiE = MPQYiE.subs(Q, -solve(WE, Q)[0])
MPWYiE

Eq(P_{marg}, P_0*(-y*sqrt(1/w) + y + y_{int}*sqrt(1/w))**2/(y_{int}**2*sqrt(1/w)))

We then express $P_0$ as a function of $w$ (and the other paramters), defining the 1-dimensional curve in $P_0, w$ space that any active active reparamterization must fulfil. 

In [60]:
P0WYiE = Eq(P, solve(MPWYiE, P)[0])
P0WYiE

Eq(P_0, P_{marg}*y_{int}**2*sqrt(1/w)/(-y*sqrt(1/w) + y + y_{int}*sqrt(1/w))**2)

Uncomment the below to obtain the closed form solution for the inverse equation $w=w(P_0)$

In [61]:
#WP0YiE = Eq(w, solve(P0WYiE,w)[0].simplify())
#WP0YiE

For the inequality, we first note that $P_0\propto P_{marg}$. We now need to distinguish the following two cases (note that the price is always quoted as numeraire / risk asset) that ensure that the AMM either matches the market price, or it is _out of the money_, ie not willing to trade at prevailing prices.

1. **AMM sells risk asset.** If the AMM sells the risk asset it is only willing to sell it at market _or higher_, ie $P_{marg} \geq P_{market}$

1. **AMM buys risk asset.** If the AMM buys the risk asset it is only willing to buy it at market _or lower_, ie $P_{marg} \leq P_{market}$

## Creating the Formulas object for export

The `FORMULAS` object contains all key formulas of this worksheet, and it allows to easily import and use them in other workbooks. 

In [62]:
try:
    from .formulalib import Formulas, __VERSION__ as _fversion, __DATE__ as _fdate
except:
    from formulalib import Formulas, __VERSION__ as _fversion, __DATE__ as _fdate
FORMULAS = Formulas(version=__VERSION__,date=__DATE__)
print(f"formulalib Version v{_fversion} ({_fdate})")
print(f"FORMULAS Version v{FORMULAS.version} ({FORMULAS.date})")

formulalib Version v2.2 (20/Nov/2022)
FORMULAS Version v3.1+margp2+TODO (12/Dec/2022)


### Core equations

In [63]:
FORMULAS.add(
    "PIE", PIE, 
    "Pool Invariant Equation (risk asset)", 
    "expressed as y as a function of x and parameters P, x0, Gamma", 
)

Eq(y, P_0*x_0*(x*(\Gamma - 1) - x_0*(\Gamma - 2))/(\Gamma*x - x_0*(\Gamma - 1)))

In [64]:
FORMULAS.add(
    "PINE", PINE, 
    "Pool Invariant Equation (numeraire)", 
    "expressed as y as a function of x and parameters P, y0, Gamma", 
)

Eq(y, y_0*(P_0*x*(\Gamma - 1) - y_0*(\Gamma - 2))/(P_0*\Gamma*x - y_0*(\Gamma - 1)))

In [65]:
FORMULAS.add(
    "PIQE", PIQE, 
    "Pool Invariant Equation (Q form)", 
    "expressed as Q equals function of x,y with the intercepts as parameters", 
)

Eq(Q, x*y/((x - x_{int})*(y - y_{int})))

In [66]:
FORMULAS.add(
    "PIQ2E", PIQ2E, 
    "Pool Invariant Equation (Q form)", 
    "expressed as Q equals function of x,y with the intercepts as parameters (alternitive form)", 
)

Eq(Q*(x - x_{int})*(y - y_{int}), x*y)

In [67]:
FORMULAS.add(
    "PIRE", PIRE, 
    "Pool Invariant Equation (r form)", 
    "expressed as y as a function of r and parameters P, x0, Gamma", 
)

Eq(y, -P_0*r*x*(-\Gamma + r*(\Gamma - 2) + 1)/(\Gamma - r*(\Gamma - 1)))

In [68]:
FORMULAS.add(
    "PIRIE", PIRIE, 
    "Pool Invariant Equation (rho)", 
    "expressed as y as a function of rho and parameters P, x0, Gamma", 
)

Eq(y, P_0*x_0*(-\Gamma + \rho*(\Gamma - 1) + 2)/(\Gamma*\rho - \Gamma + 1))

In [69]:
FORMULAS.add(
    "PIA1E", PIA1E, 
    "Pool Invariant Equation (asymptotic)", 
    "expressed as hyperbola equation with asymptotes and kappa", 
)

Eq((x - x_{asym})*(y - y_{asym}), \kappa)

In [70]:
FORMULAS.add(
    "SE", SE, 
    "Swap Equation", 
    "expressed as minus Delta y as a function of Delta x and all other parameters and variables from the PIE", 
)

Eq(-\Delta{y}, P_0*\Delta{x}*x_0**2/((\Gamma*(x - x_0) + x_0)*(\Gamma*(\Delta{x} + x - x_0) + x_0)))

In [71]:
FORMULAS.add(
    "SRE", SRE, 
    "Swap Equation (r)", 
    "expressed as minus Delta y as a function of Delta x and all other parameters and variables from the PIRE", 
)

Eq(\Delta{y}, P_0*\Delta{x}*r**2*x/((\Gamma*(r - 1) - r)*(\Gamma*(\Delta{x} - r*x + x) + r*x)))

In [72]:
FORMULAS.add(
    "SRIE", SRIE, 
    "Swap Equation (rho)", 
    "expressed as minus Delta y as a function of Delta x and all other parameters and variables from the PIRIE", 
)

Eq(-\Delta{y}, P_0*\Delta{x}*x_0/((\Gamma*(\Delta{x} + x_0*(\rho - 1)) + x_0)*(\Gamma*(\rho - 1) + 1)))

In [73]:
FORMULAS.add(
    "MSE", MSE, 
    "Marginal Swap Equation", 
    "the Swap Equation for Delta x -> dx; dx is brought to the LHS", 
)

Eq(-dy/dx, P_0*x_0**2/(\Gamma*(x - x_0) + x_0)**2)

In [74]:
FORMULAS.add(
    "MPE", MPE, 
    "Marginal Price Equation", 
    "this is the same as the Marginal Swap Equation, except that the LHS is recognized as the marginal price", 
)

Eq(P_{marg}, P_0*x_0**2/(\Gamma*(x - x_0) + x_0)**2)

In [75]:
FORMULAS.add(
    "MPERev", MPERev, 
    "Reverse Marginal Price Equation", 
    "Marginal price eqaution seen from the other side (ie based on y, y0)", 
)

Eq(P_{marg}, P_0*(\Gamma*(y - y_0) + y_0)**2/y_0**2)

In [76]:
FORMULAS.add(
    "MPQXiE", MPQXiE, 
    "Marginal Price Equation (Q,xint)", 
    "Reparametrization of the MPE as a function of $Q, x_{int}$", 
)

Eq(P_{marg}, P_0*Q*x_{int}**2/(-Q*x + Q*x_{int} + x)**2)

In [77]:
FORMULAS.add(
    "MPQYiE", MPQYiE, 
    "Marginal Price Equation (Q,yint)", 
    "Reparametrization of the MPE as a function of $Q, y_{int}$", 
)

Eq(P_{marg}, P_0*(-Q*y + Q*y_{int} + y)**2/(Q*y_{int}**2))

In [78]:
FORMULAS.add(
    "MPWYiE", MPWYiE, 
    "Marginal Price Equation (w,yint)", 
    "Reparametrization of the MPE as a function of $w, y_{int}$", 
)

Eq(P_{marg}, P_0*(-y*sqrt(1/w) + y + y_{int}*sqrt(1/w))**2/(y_{int}**2*sqrt(1/w)))

In [79]:
FORMULAS.add(
    "MPWYiE", MPWYiE, 
    "Marginal Price Equation (w,yint)", 
    "Reparametrization of the MPE as a function of $w, y_{int}$", 
)

Eq(P_{marg}, P_0*(-y*sqrt(1/w) + y + y_{int}*sqrt(1/w))**2/(y_{int}**2*sqrt(1/w)))

In [80]:
FORMULAS.add(
    "P0WYiE", P0WYiE, 
    "Constant Marginal Price Curve Equation", 
    "Establishes relationship $P_0=P_0(w)$ for which the marginal price remains const", 
)

Eq(P_0, P_{marg}*y_{int}**2*sqrt(1/w)/(-y*sqrt(1/w) + y + y_{int}*sqrt(1/w))**2)

In [81]:
FORMULAS.add(
    "PLE", PLE, 
    "Pool leverage equation", 
    "Establishes relationship between kappa, k, Gamma", 
)

Eq(\kappa, k/\Gamma**2)

In [82]:
FORMULAS.add(
    "KPYDE", KPYDE, 
    "Alternative definition equation for kappa", 
    "Defines kappa in terms of x0, y0, Gamma$", 
)

Eq(\kappa, x_0*y_0/\Gamma**2)

In [83]:
FORMULAS.add(
    "GSE", GSE, 
    "Gamma scaling equation", 
    "Defines the scaling factor 1/Gamma as function of the range size Py/Px", 
)

Eq(sqrt(\kappa)/sqrt(k), 1/(1 - (P_x/P_y)**(1/4)))

In [84]:
FORMULAS.add(
    "MPRE", MPRE, 
    "Marginal Price Equation (fixed x/x0)", 
    "this is the same as the Marginal Swap Equation, except that the LHS is recognized as the marginal price; x/x0 = r is fixed", 
)

Eq(P_{marg}, P_0*r**2/(\Gamma*(r - 1) - r)**2)

In [85]:
FORMULAS.add(
    "MPRIE", MPRIE, 
    "Marginal Price Equation (fixed x0/x)", 
    "this is the same as the Marginal Swap Equation, except that the LHS is recognized as the marginal price; x/x0 = r is fixed", 
)

Eq(P_{marg}, P_0/(\Gamma*(\rho - 1) + 1)**2)

### B, S equations

In [86]:
FORMULAS.add(
    "BDE", BDE, 
    "B definition equation", 
    "This equation defines the parameter B = sqrt Px", 
)

Eq(B, sqrt(P_x))

In [87]:
FORMULAS.add(
    "SDE", SDE, 
    "S definition equation", 
    "This equation defines the parameter S = sqrt Py - sqrt Px", 
)

Eq(S, -sqrt(P_x) + sqrt(P_y))

In [88]:
FORMULAS.add(
    "MPBSE", MPBSE, 
    "Marginal Price Equation in B,S", 
    "The marginal price equation using B,S parameterization", 
)

Eq(P_{marg}, (B + S*y/y_{int})**2)

In [89]:
FORMULAS.add(
    "YMPE", YMPE, 
    "Marginal Price Equation solved for y", 
    "The marginal price equation using B,S parameterization, solved for y", 
)

Eq(y, y_{int}*(-B + sqrt(P_{marg}))/S)

### Definition equations and other relationships

In [90]:
FORMULAS.add(
    "QGDE", QGDE, 
    "Q Gamma Definition Equation", 
    "Gamma as a function of Q", 
)

Eq(Q, (1 - \Gamma)**2)

In [91]:
FORMULAS.add(
    "GQE", GQE, 
    "Gamma Q Equation", 
    "Gamma as a function of Q", 
)

Eq(\Gamma, 1 - sqrt(Q))

In [92]:
FORMULAS.add(
    "GFQE", GFQE, 
    "Gamma Q Formula Equation", 
    "an important relationship between functions of Gamma and Q", 
)

Eq((2 - \Gamma)/(1 - \Gamma), 1 + 1/sqrt(Q))

In [93]:
FORMULAS.add(
    "XiDE", XiDE, 
    "Xint Definition Equation", 
    "xint as function of x0 and Gamma", 
)

Eq(x_{int}, x_0*(2 - \Gamma)/(1 - \Gamma))

In [94]:
FORMULAS.add(
    "YiDE", YiDE, 
    "Yint Definition Equation", 
    "xint as function of y0 and Gamma", 
)

Eq(y_{int}, y_0*(2 - \Gamma)/(1 - \Gamma))

In [95]:
FORMULAS.add(
    "XiQE", XiQE, 
    "Xint Q Equation", 
    "xint as function of x0 and Q", 
)

Eq(x_{int}, x_0*(1 + 1/sqrt(Q)))

In [96]:
FORMULAS.add(
    "YiQE", YiQE, 
    "Yint Q Equation", 
    "yint as function of y0 and Q", 
)

Eq(y_{int}, y_0*(1 + 1/sqrt(Q)))

In [97]:
FORMULAS.add(
    "PxDE", PxDE, 
    "Px Equation", 
    "Px as function of P and Q", 
)

Eq(P_x, P_0*Q)

In [98]:
FORMULAS.add(
    "PyDE", PyDE, 
    "Py Equation", 
    "Py as function of P and Q", 
)

Eq(P_y, P_0/Q)

In [99]:
FORMULAS.add(
    "PXYE", PXYE, 
    "PXY Equation", 
    "P as a function of x0 and y0", 
)

Eq(P_0, y_0/x_0)

In [100]:
FORMULAS.add(
    "PXYiE", PXYiE, 
    "PXYi Equation", 
    "P as a function of xint and yint", 
)

Eq(P_0, y_{int}/x_{int})

In [101]:
FORMULAS.add(
    "QPE", QPE, 
    "QP Equation", 
    "Q as a function of Px and Py", 
)

Eq(Q, sqrt(P_x/P_y))

In [102]:
FORMULAS.add(
    "WDE", WDE, 
    "Range width definition equation", 
    "w as ratio Py/Px ", 
)

Eq(w, P_y/P_x)

In [103]:
FORMULAS.add(
    "WE", WE, 
    "Range width equation", 
    "Py/Px as a function of Q", 
)

Eq(w, Q**(-2))

In [104]:
FORMULAS.add(
    "PPE", PPE, 
    "PP Equation", 
    "P as a function of Px and Py", 
)

Eq(P_0, sqrt(P_x*P_y))

In [105]:
FORMULAS.add(
    "XiPE", XiPE, 
    "Xint from P", 
    "xint as function of Px and Py", 
)

Eq(x_{int}, y_{int}/sqrt(P_x*P_y))

In [106]:
FORMULAS.add(
    "YiPE", XiPE, 
    "yint from P", 
    "yint as function of Px and Py", 
)

Eq(x_{int}, y_{int}/sqrt(P_x*P_y))

In [107]:
FORMULAS.add(
    "XADE", XADE, 
    "Definition equation for xasym", 
    "Defines $x_{asym}$ in terms of x0, Gamma", 
)

Eq(x_{asym}, x_0*(1 - 1/\Gamma))

In [108]:
FORMULAS.add(
    "YADE", YADE, 
    "Definition equation for yasym", 
    "Defines $y_{asym}$ in terms of y0, Gamma", 
)

Eq(y_{asym}, y_0*(1 - 1/\Gamma))

In [109]:
FORMULAS.add(
    "YAXDE", YAXDE, 
    "Alternative definition equation for yasym", 
    "Defines $y_{asym}$ in terms of x0, P0, Gamma", 
)

Eq(y_{asym}, P_0*x_0*(1 - 1/\Gamma))

In [110]:
FORMULAS.add(
    "KPDE", KPDE, 
    "Definition equation for kappa$", 
    "Defines $\kappa$ in terms of x0, P0, Gamma", 
)

Eq(\kappa, P_0*x_0**2/\Gamma**2)

In [111]:
FORMULAS.add(
    "RXE", RXE, 
    "Ratio equation (x/x0)", 
    "Defines r as the ratio x/x0", 
)

Eq(x_0, r*x)

In [112]:
FORMULAS.add(
    "RXIE", RXIE, 
    "Inverse ratio equation (x0/x)", 
    "Defines rho as the ratio x0/x", 
)

Eq(x, \rho*x_0)